In [1]:
# CONSTANTS
salary_cap = 94143000

In [2]:
import urllib.request
from nba_py import player, team
from nba_py.constants import *
from bs4 import BeautifulSoup
from __future__ import print_function
import pandas as pd
import numpy as np
import re
import itertools
import datetime as dtime

In [3]:
def strip_name(name):
    return re.sub(r'[^\w\s]','',name.lower())

In [4]:
def create_salary_csv():
    salaryDT = pd.read_csv('data/contracts_players_contracts.csv')
    salaryDT['formatted_name'] = salaryDT.apply(lambda row: strip_name(row['Player']), axis=1)
    return salaryDT

In [5]:
def find_valid_team_players(team_name, valid_players):
    teamDT = team.TeamCommonRoster(team_id=TEAMS[team_name]['id'],season='2016-17').roster()
    roster = list(teamDT['PLAYER'])
    rookies = 0
    a = list(roster)
    
    for player in a:
        if(len(salaryDT[salaryDT['formatted_name'] == strip_name(player)]) != 0):
            if(salaryDT[salaryDT['formatted_name'] == strip_name(player)]['Signed Using'].iloc[0] == '1st Round Pick'):
                rookies = rookies + 1
                roster.remove(player)
        else:
            roster.remove(player)
        
    for _ in range(5-rookies):
        team_contracts = salaryDT[salaryDT.Player.isin(roster)][['Player','2016-17']]
        roster.remove(team_contracts.iloc[0]['Player'])
    
    valid_players.append(roster)
        
    return valid_players

In [6]:
def find_valid_players():
    valid_players = []
    for team_name in list(TEAMS.keys()):
        valid_players = find_valid_team_players(team_name, valid_players)
    valid_players = list(itertools.chain.from_iterable(valid_players))
    
    return valid_players

In [7]:
def create_master_player_DT():
    teamDTList = []
    for team_name in list(TEAMS.keys()):
        teamDT = team.TeamCommonRoster(team_id=TEAMS[team_name]['id'],season='2016-17').roster()
        teamDTList.append(teamDT)
    return pd.concat(teamDTList)

In [8]:
# CREATE masterPlayerDT, filter out any rookies/invalid players to choose
salaryDT = create_salary_csv()

valid_players = find_valid_players()

masterPlayerDT = create_master_player_DT()
masterPlayerDT = masterPlayerDT[masterPlayerDT['EXP'] != "R"]
masterPlayerDT = masterPlayerDT[masterPlayerDT.PLAYER.isin(valid_players)]

In [9]:
def get_data(url_string):
    soup = BeautifulSoup(urllib.request.urlopen(url_string).read(), 'lxml')
    vorp = 0
    try:
        result_row = soup.find(id = 'advanced.2016').find_all('td')
        vorp_element = result_row[len(result_row)-1].get_text()
        vorp = float(vorp_element)
    except:
        pass
    return vorp

In [10]:
def find_player_vorp(player_name):
    base_url = 'http://www.basketball-reference.com/players/'
    name_array = strip_name(player_name).split(' ')
    first_name = name_array[0]
    last_name = name_array[len(name_array)-1]
    url_string = base_url + last_name[0] + '/' + last_name[0:4] + first_name[0:1] + '01.html'
    vorp = get_data(url_string)
    return vorp

In [11]:
def create_vorp_array(player_names):
    vorp_array = np.zeros(len(player_names))
    for index, player_name in enumerate(player_names):
        vorp_array[index] = find_player_vorp(player_name)
    return vorp_array

In [ ]:
masterPlayerDT['VORP'] = pd.Series(create_vorp_array(masterPlayerDT['PLAYER']), index=masterPlayerDT.index)